**분석개요**

    A. 꾸온꾸 성과 분석 리포트
        1. 매체별 유입 분석
            1) 유입
            2) 재방문
            3) 구독 전환
            4) 이상 유저
            5) 앱설치
            6) 활성
            7) 리텐션
        2. 이동 경로 분석

In [1]:
# basic
import gc
import os
import sys
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#--------------------#
# handling
#--------------------#
import math
import time
import random
# import openpyxl
import importlib
import xlsxwriter
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime, timedelta
from scipy.stats import pearsonr
import difflib

#--------------------#
# Vis
#--------------------#
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
## Vis for jupyter theme
from IPython.display import Image
from jupyterthemes import jtplot
jtplot.style(theme= 'grade3', context='notebook', ticks=True, grid=False) ## dark backgroud jupyter notebook theme
# plt.style.use(['seaborn-white']) ## dark background style ## white style : 'seaborn-white'
plt.style.use(['dark_background']) ## dark background style ## white style : 'seaborn-white'
## Vis for korean 
import matplotlib.font_manager as fm
font_location = '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf'
fprop = fm.FontProperties(fname=font_location)
font_name = fprop.get_name()
matplotlib.rc('font', family=font_name)

In [2]:
#--------------------#
# sphere package
#--------------------#
sys.path.append("/home/das_share/sphere_class/")
import SpherePackage
from SpherePackage import *
for pkg in [SpherePackage] :
    _ = importlib.reload(pkg)

#--------------------#
# kto package
#--------------------#
sys.path.append("../src/")
import kto_config
import kto_util
import kto_prep
from kto_config import *
from kto_prep import *
from kto_util import *
from run_cdj import *

for pkg in [kto_config, kto_prep, kto_util] :
    _ = importlib.reload(pkg)

#-------------------#
# pickle
#-------------------#
sys.path.append("/home/minkyung62/analysis_report/00_custom_analysis/kto_custom/notebook_git/return_pickle.py")
import return_pickle
from return_pickle import *
for pkg in [return_pickle] :
    _ = importlib.reload(pkg)

# 1.Data Import

In [3]:
#------------------------------------------------#
# 1. log Data import
#------------------------------------------------#
df_app_log = ReadFile.read_pickle(e_date='20221120', s_date='20221020')
df_app_log = SpherePrep.BasicPrep.basic_prep(df_app_log)   

df_app_log.head(3)

,date,duration,sid,seq,app_key,sphere_id,user_id,uid,adid,gender,birth_y,user_grade,created_date,events,referrer,install_referrer,session,abs_events,day
3333010,2022-10-20 00:00:02,1680.607,gVACp95v8aSXECgOX6EWNq,1,qsJfZhE2Ft85SvZnBo2KSL,nT4C7Uf3s4_ziY--3_qsWZpngOloj,,uQZs-0CPowzdFzl3BPTRkQ,,,0,,2022-10-20 00:00:02,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '211.11...","[sapContentDetail, subscribeClicsapContentShar...",2022-10-20
3332913,2022-10-20 00:00:02,2.832,bgON0628_pX07M7zqjQFGm,1,qsJfZhE2Ft85SvZnBo2KSL,jxJXzaDhlSQgbRy1_VDg7yKQj7RKx,,KcL2M2Ab8lS_gd24_KbXbg,,,0,,2022-10-20 00:00:02,"[{'name': 'contentMain', 'timestamp': 16661916...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Spider'}, 'ip': '211.249...",[contentMain],2022-10-20
3333017,2022-10-20 00:00:02,0.705,Er6klS37BPU4FacYHdCxeI,1,qsJfZhE2Ft85SvZnBo2KSL,s7YQZN3kmOOw2Q9Bok1EVWo3nmX3G,,RkCFnnvZi-x-OUhoz8G-6A,,,0,,2022-10-20 00:00:02,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': 'NAVER_GFA', 'utm_campaign': '2...",[],"{'device': {'brand': 'Apple'}, 'ip': '110.70.5...",[sapContentDetail],2022-10-20


# 2.Preprocess

In [5]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
# ## 2) user prop
df_prop = SpherePrep.Prop.df_prop_pipe(df_app_log, KEY_ID = [KEY_ID_DEVICE, KEY_ID_USER], drop_none_prop=False)       # user data

## 3) param - 1
_, df_param_all = DataImport.json_to_dataframe_nodeN(
    df_app_log, [KEY_ID_DEVICE, KEY_ID_USER]
    )

# ## 3) param - 2
# _, df_param_all_session = json_to_dataframe_nodeN_letter( 
#     df_app_log, [KEY_ID_DEVICE, KEY_ID_USER, 'sid']
#     ) ### + param by session

## 4) referr
lst_extrated_referr_cols = ['utm_campaign', 'utm_source', 'utm_medium']
df_app_log = SpherePrep.Attribution_Prep.gen_session_referr_cols(
        df_app_log, 
        referr_col = 'referrer', 
        lst_extrated_referr_cols = lst_extrated_referr_cols,
        fill_na_organic = True)

# 3.Utils

In [6]:
# cnt -> percent function
def cnt_to_percent_ver1(df, col):
    df[col] = df[col].astype(float)

    if  df.index.dtype != 'str':
        df.index = df.index.astype('str')
    
    _lst = []
    for i in range(0,len(df)):
        x = round((df[col][i] / df[col].sum())*100, 2)
        _lst.append(x)

    df['percent']= _lst
    return df


def get_daily_cnt(
                  dict_output, lst, nm, dict_input, df_target = df_app_log, KEY_ID = KEY_ID_DEVICE, 
                  make_sub_dict = False):
    # 1. 전체 일별 수치
    _df_target_daily = df_target[df_target[KEY_ID].isin(lst)].groupby('day')[KEY_ID]\
                    .nunique().to_frame(name = f'total_{nm}')

    # 2. 매체별 일별 수지
    for key, values in dict_input.items():
        _lst_id = list(set(lst) &  set(values))
        _df = df_target[df_target[KEY_ID].isin(_lst_id)].groupby('day')[KEY_ID]\
                    .nunique().to_frame(name = f'{key}_{nm}').fillna(0)
        if make_sub_dict == True:
            dict_output[f'{key}'] = {}

        dict_output[f'{key}'][f'{nm}'] = _df
        # dict_output[f'{key}_{nm}'] 
        _df_target_daily[f'{key}'] = _df[f'{key}_{nm}']
        _df_target_daily.fillna(0, inplace=True)
        # pd.concat([_df_target_daily, dict_output[f'{key}'][f'{key}_{nm}']], axis=1).fillna(0)
        
    return _df_target_daily

# 4.Analysis
## 0)Segment Setting

In [7]:
#------------------------------------------------#
# 1. segment by referrer
#------------------------------------------------#
dict_lst_user_from_referrs = KtoPrep.get_user_list_by_referr(
    df_app_log, KEY_ID = KEY_ID_DEVICE,
    DEPTH_1_COL = 'referrer_utm_campaign', 
    DEPTH_2_COL = 'referrer_utm_source') 
dict_lst_user_from_referrs_gguonggu = dict_lst_user_from_referrs['Travel_look']
dict_lst_user_from_referrs_gguonggu['organic']  = dict_lst_user_from_referrs['organic']['organic'] # organic 유입 추가

#------------------------------------------------#
# 2. segment by duration per device
#------------------------------------------------#
df_duration_per_device = df_app_log.groupby([KEY_ID_DEVICE])['duration'].sum()
dict_duration_seg = dict()
dict_duration_seg['high'] = df_duration_per_device[
    df_duration_per_device >= df_duration_per_device.quantile(0.66)].index
dict_duration_seg['middle'] = df_duration_per_device[
    (df_duration_per_device < df_duration_per_device.quantile(0.66)) &
    (df_duration_per_device >= df_duration_per_device.quantile(0.33))
    ].index
dict_duration_seg['low'] = df_duration_per_device[
    df_duration_per_device < df_duration_per_device.quantile(0.33)].index

#------------------------------------------------#
# 3. segment by new or re
#------------------------------------------------#
## 아래 코드가 실행되지 않아 created_date를 datetime64으로 형변환 시도 
# df_app_log['created_date'] = pd.DatetimeIndex(df_app_log['created_date'])
lst_new_device = list(df_app_log[df_app_log['created_date'] >= datetime.strptime('20221020', '%Y%m%d')][KEY_ID_DEVICE].unique())
lst_re_device = list(df_app_log[df_app_log['created_date'] < datetime.strptime('20221020', '%Y%m%d')][KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 4. segment by fraud or total
#------------------------------------------------#
lst_fraud_device = list(df_duration_per_device[df_duration_per_device < 5].index)
lst_total_device = list(df_app_log[KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 5. segment by demo (gender or age)
#------------------------------------------------#
# ## gender
# dict_seg_gender = dict()
# dict_seg_gender['m'] = list(df_prop[df_prop['gender'] == 'm'][KEY_ID_DEVICE])
# dict_seg_gender['f'] = list(df_prop[df_prop['gender'] == 'f'][KEY_ID_DEVICE])

# ## age
# dict_seg_age = dict()
# # dict_seg_age['20_30'] = list(df_prop[(datetime.today().year - df_prop['birth_y']) < 40][KEY_ID_DEVICE])
# # dict_seg_age['40_50'] = list(df_prop[(datetime.today().year - df_prop['birth_y']) >= 40][KEY_ID_DEVICE])
# # # # ======================================================== #  next version
# dict_seg_age['20_30'] = list(df_prop[df_prop['age'] < 40][KEY_ID_DEVICE])
# dict_seg_age['40_50'] = list(df_prop[df_prop['age'] >= 40][KEY_ID_DEVICE])

In [8]:
#------------------------------------------------#
# 6. prep referr 
## ver 1: original
## ver 2: 'kto_ins', 'kto_FB', 'kto_kastory', 'kto_twt', 'kto_blog'  --> 'kto_sns'
#------------------------------------------------#

## before prep
dict_lst_user_from_referrs_gguonggu.keys()

## prep
dict_lst_user_from_referrs_gguonggu2 = dict_lst_user_from_referrs_gguonggu.copy()

dict_lst_user_from_referrs_gguonggu2['kto_sns'] =\
    list(
        set(dict_lst_user_from_referrs_gguonggu2['kto_kastory'] + dict_lst_user_from_referrs_gguonggu2['kto_ins'] + dict_lst_user_from_referrs_gguonggu2['kto_FB'] + 
            dict_lst_user_from_referrs_gguonggu2['kto_twt'] + dict_lst_user_from_referrs_gguonggu2['kto_blog']) 
        )

del dict_lst_user_from_referrs_gguonggu2['kto_kastory']
del dict_lst_user_from_referrs_gguonggu2['kto_ins']
del dict_lst_user_from_referrs_gguonggu2['kto_FB']
del dict_lst_user_from_referrs_gguonggu2['kto_twt']
del dict_lst_user_from_referrs_gguonggu2['kto_blog']

## after prep
dict_lst_user_from_referrs_gguonggu2.keys()

dict_keys(['FB_INSTA', 'KAKAO_moment', 'everytime', 'NAVER_GFA', 'kto_ins', 'kto_FB', 'kto_kastory', 'kto_twt', 'kto_blog', 'organic'])

dict_keys(['FB_INSTA', 'KAKAO_moment', 'everytime', 'NAVER_GFA', 'organic', 'kto_sns'])

## 1)매체 성과 분석
### (1)유입
- 전체 유입
- 일별 유입
    - 전체 유입
    - 신규 유입
    - 기존 유입

In [9]:
#########################################
# 1. 매체별 전체 유입인원
## 전체 / 일별 총 유입인원
#########################################
dict_output_inflow_total= {}
dict_output_inflow_daily= {}
dict_output_inflow_total2= {}
dict_output_inflow_daily2= {}

def count_inflow(dict_input, dict_output_total, dict_output_daily, KEY_ID = KEY_ID_DEVICE, lst = list(df_app_log[KEY_ID_DEVICE].unique()), nm = 'inflow'):

    _df_target_daily = df_app_log[
                        df_app_log[KEY_ID].isin(lst)].groupby('day')[KEY_ID]\
                            .nunique().to_frame(name = f'total_{nm}')

    for _referrer, _segment in dict_input.items():
        ## 1) 전체 총 유입인원
        dict_output_total[f'{_referrer}'] = {}
        dict_output_total[f'{_referrer}'][f'{nm}'] = len(set(_segment))

        ## 2) 일별 총 유입인원
        _lst_segment = list(set(lst) &  set(_segment))
        _df = df_app_log[df_app_log[KEY_ID].isin(_lst_segment)].groupby('day')[KEY_ID].nunique().to_frame(name = f'{_referrer}').fillna(0)
        
        dict_output_daily[f'{_referrer}'] = {}
        dict_output_daily[f'{_referrer}'][f'{nm}'] = _df
        _df_target_daily[f'{_referrer}'] = _df[f'{_referrer}']
        _df_target_daily.fillna(0, inplace=True)


    ### 1-2) 전체 총 유입인원 df화
    _df_output = pd.DataFrame.from_dict(dict_output_total, orient='index', columns=['inflow'])
    _df_output = cnt_to_percent_ver1(_df_output, 'inflow')
    _df_output = _df_output.sort_values(by = 'inflow', ascending=False)

    return _df_output, _df_target_daily

_df_inflow_total, _df_inflow_daily = count_inflow(dict_lst_user_from_referrs_gguonggu, dict_output_inflow_total, dict_output_inflow_daily)
_df_inflow_total2, _df_inflow_daily2 = count_inflow(dict_lst_user_from_referrs_gguonggu2, dict_output_inflow_total2, dict_output_inflow_daily2)

print('ver1')
_df_inflow_total
print('ver2')
_df_inflow_total2
pd.set_option('max_rows', 100)
print('ver1')
_df_inflow_daily
print('ver2')
_df_inflow_daily2

ver1


,inflow,percent
organic,1865431.0,99.08
FB_INSTA,8329.0,0.44
KAKAO_moment,5060.0,0.27
NAVER_GFA,1893.0,0.10
everytime,1736.0,0.09
kto_ins,164.0,0.01
kto_kastory,84.0,0.00
kto_blog,49.0,0.00
kto_twt,11.0,0.00
kto_FB,9.0,0.00


ver2


,inflow,percent
organic,1865431.0,99.08
FB_INSTA,8329.0,0.44
KAKAO_moment,5060.0,0.27
NAVER_GFA,1893.0,0.10
everytime,1736.0,0.09
kto_sns,317.0,0.02


ver1


,total_inflow,FB_INSTA,KAKAO_moment,everytime,NAVER_GFA,kto_ins,kto_FB,kto_kastory,kto_twt,kto_blog,organic
day,,,,,,,,,,,
2022-10-20,67792,1.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,63632
2022-10-21,73371,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,70475
2022-10-22,93592,2.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,90456
2022-10-23,83872,1.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,78990
2022-10-24,68505,2.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,64271
2022-10-25,64631,1.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,59618
2022-10-26,60008,7.0,2.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,56712
2022-10-27,50875,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,46902
2022-10-28,74171,3.0,2.0,0.0,1.0,4.0,0.0,0.0,1.0,0.0,70608


ver2


,total_inflow,FB_INSTA,KAKAO_moment,everytime,NAVER_GFA,organic,kto_sns
day,,,,,,,
2022-10-20,67792,1.0,2.0,0.0,1.0,63632,2.0
2022-10-21,73371,0.0,1.0,1.0,2.0,70475,1.0
2022-10-22,93592,2.0,0.0,1.0,2.0,90456,1.0
2022-10-23,83872,1.0,0.0,0.0,1.0,78990,4.0
2022-10-24,68505,2.0,4.0,0.0,0.0,64271,4.0
2022-10-25,64631,1.0,2.0,0.0,0.0,59618,3.0
2022-10-26,60008,7.0,2.0,0.0,2.0,56712,3.0
2022-10-27,50875,1.0,1.0,1.0,2.0,46902,1.0
2022-10-28,74171,3.0,2.0,0.0,1.0,70608,5.0


### (2)재방문
- 방문 빈도별 재방문율
- 주차별 재방문율

In [23]:
#------------------------------------------------#
## 2. 재방문율
#------------------------------------------------#
### 1) 1회 / 2회 이상 / 3회 이상 방문자
dict_output_revisit = dict()
dict_output_revisit2 = dict()


#### count revisit rate
def revisit_rate(dict_input, dict_output):
    for _referrer, _segment in dict_input.items():
        dict_output[_referrer] = dict()
        _df_user_day_cnt = df_app_log[df_app_log[KEY_ID_DEVICE].isin(_segment)].groupby(KEY_ID_DEVICE)['day'].count()

        only_1_visit_rate = round(_df_user_day_cnt.value_counts()[1] / len(_segment), 4)
        re_visit_rate = round(1 - only_1_visit_rate, 4)
        re_visit_over_3_rate = round(1 - _df_user_day_cnt.value_counts()[:2].sum() / len(_segment), 4)
        
        dict_output[_referrer]['only_1_visit_rate'] = only_1_visit_rate
        dict_output[_referrer]['re_visit_rate'] = re_visit_rate
        dict_output[_referrer]['re_visit_over_3_rate'] = re_visit_over_3_rate

    _df_output = pd.DataFrame.from_dict(
        dict_output, orient = 'index'
        ).sort_values(by = 're_visit_rate', ascending=False).reset_index()
    
    return _df_output

df_ouput_revisit = revisit_rate(dict_lst_user_from_referrs_gguonggu, dict_output_revisit)
df_ouput_revisit2 = revisit_rate(dict_lst_user_from_referrs_gguonggu2, dict_output_revisit2)

print('ver1')
df_ouput_revisit
print('ver2')
df_ouput_revisit2

ver1


,index,only_1_visit_rate,re_visit_rate,re_visit_over_3_rate
0,kto_ins,0.4146,0.5854,0.4512
1,kto_FB,0.4444,0.5556,0.3333
2,kto_twt,0.5455,0.4545,0.1818
3,kto_blog,0.6327,0.3673,0.2449
4,organic,0.8667,0.1333,0.0665
5,KAKAO_moment,0.8798,0.1202,0.0322
6,kto_kastory,0.8810,0.1190,0.0476
7,everytime,0.9044,0.0956,0.0150
8,FB_INSTA,0.9301,0.0699,0.0136
9,NAVER_GFA,0.9308,0.0692,0.0174


ver2


,index,only_1_visit_rate,re_visit_rate,re_visit_over_3_rate
0,kto_sns,0.5773,0.4227,0.3060
1,organic,0.8667,0.1333,0.0665
2,KAKAO_moment,0.8798,0.1202,0.0322
3,everytime,0.9044,0.0956,0.0150
4,FB_INSTA,0.9301,0.0699,0.0136
5,NAVER_GFA,0.9308,0.0692,0.0174


### (3)전환
: 랜딩페이지에서 콘테스트 페이지까지의 전환 KPI 분석
- 전환수
- 전환율

In [11]:
#########################################
# 1. 구독완료
## 1) 일별 구독완료 유저 수
## 2) 구독전환율
#########################################
lst_landing_device = list(df_param_all[KEY_ID_DEVICE][df_param_all['params_sapContentName'].str.contains('기온별_여행코디', na=False)].unique())
lst_contest_device = df_param_all[KEY_ID_DEVICE][df_param_all['params_sapContentName'].str.contains('콘테스트!', na=False)]

dict_output_conversion = {}
df_daily_contest = pd.DataFrame()
df_kpi_conversion = pd.DataFrame(index=[
                                'landing_cnt', 
                                'contest_cnt', 
                                'conversion'])
dict_output_conversion2 = {}
df_daily_contest2 = pd.DataFrame()
df_kpi_conversion2 = pd.DataFrame(index=[
                                'landing_cnt', 
                                'contest_cnt', 
                                'conversion'])

def calculate_conversion(dict_input, dict_output, KPI, df_output_conversion, KEY_ID = KEY_ID_DEVICE):
    _df_param_target = df_param_all[df_param_all['params_sapContentName'].str.contains('콘테스트!', na=False)]

    df_daily_output = _df_param_target.groupby('day')[KEY_ID_DEVICE].nunique().to_frame(name = 'total_cnt')
    _lst_device = list(_df_param_target[KEY_ID_DEVICE].unique())

    for _referrer, _segment in dict_input.items():
        ### 2) 매체별 일별 콘테스트 페이지 접속 유저 수
        _lst_target = list(set(_lst_device) &  set(_segment))

        _df = _df_param_target[_df_param_target[KEY_ID_DEVICE].isin(_lst_target)].groupby('day')[KEY_ID_DEVICE].nunique().to_frame(name = f'{_referrer}' + f'_{KPI}_daily').fillna(0)

        dict_output[f'{_referrer}'] = {}
        dict_output[f'{_referrer}'][f'{KPI}_daily_cnt'] = _df
        df_daily_output[f'{_referrer}'] = _df[f'{_referrer}' + f'_{KPI}_daily']
        df_daily_output.fillna(0, inplace=True)

        ### 3) 전환율
        landing_cnt = len(_segment)
        kpi_cnt = _df_param_target[_df_param_target[KEY_ID_DEVICE].isin(_segment)][KEY_ID_DEVICE].nunique()
        dict_output[f'{_referrer}'][f'{KPI}_conversion'] = [landing_cnt,
                                                        kpi_cnt, 
                                                        (kpi_cnt/landing_cnt)]
        df_output_conversion[f'{_referrer}'] = dict_output[f'{_referrer}'][f'{KPI}_conversion']

    return df_daily_output, df_output_conversion

df_daily_kpi_cnt, df_kpi_conversion = calculate_conversion(
                                        dict_lst_user_from_referrs_gguonggu, dict_output_conversion, 
                                        KPI = 'contest', df_output_conversion=df_kpi_conversion ,KEY_ID= KEY_ID_DEVICE)

df_daily_kpi_cnt2, df_kpi_conversion2 = calculate_conversion(
                                        dict_lst_user_from_referrs_gguonggu2, dict_output_conversion2, 
                                        KPI = 'contest', df_output_conversion=df_kpi_conversion2 ,KEY_ID= KEY_ID_DEVICE)

pd.set_option('max_rows', 100)
print('ver1')
df_daily_kpi_cnt

print('ver2')
df_daily_kpi_cnt2

print('ver1')
df_kpi_conversion

print('ver2')
df_kpi_conversion2

ver1


,total_cnt,FB_INSTA,KAKAO_moment,everytime,NAVER_GFA,kto_ins,kto_FB,kto_kastory,kto_twt,kto_blog,organic
day,,,,,,,,,,,
2022-11-10,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2022-11-11,279,8.0,27.0,0.0,1.0,60.0,5.0,56.0,7.0,27.0,117
2022-11-12,169,35.0,30.0,0.0,0.0,23.0,3.0,18.0,0.0,5.0,72
2022-11-13,111,27.0,25.0,0.0,1.0,0.0,0.0,5.0,2.0,3.0,45
2022-11-14,144,26.0,27.0,1.0,0.0,3.0,1.0,0.0,1.0,6.0,77
2022-11-15,217,19.0,25.0,7.0,1.0,2.0,0.0,2.0,1.0,0.0,159
2022-11-16,124,25.0,22.0,7.0,0.0,5.0,0.0,0.0,1.0,1.0,68
2022-11-17,144,21.0,20.0,2.0,4.0,4.0,0.0,3.0,3.0,0.0,67
2022-11-18,276,21.0,18.0,7.0,1.0,70.0,0.0,0.0,0.0,3.0,207


ver2


,total_cnt,FB_INSTA,KAKAO_moment,everytime,NAVER_GFA,organic,kto_sns
day,,,,,,,
2022-11-10,3,0.0,0.0,0.0,0.0,3,0.0
2022-11-11,279,8.0,27.0,0.0,1.0,117,155.0
2022-11-12,169,35.0,30.0,0.0,0.0,72,49.0
2022-11-13,111,27.0,25.0,0.0,1.0,45,10.0
2022-11-14,144,26.0,27.0,1.0,0.0,77,11.0
2022-11-15,217,19.0,25.0,7.0,1.0,159,5.0
2022-11-16,124,25.0,22.0,7.0,0.0,68,7.0
2022-11-17,144,21.0,20.0,2.0,4.0,67,10.0
2022-11-18,276,21.0,18.0,7.0,1.0,207,73.0


ver1


,FB_INSTA,KAKAO_moment,everytime,NAVER_GFA,kto_ins,kto_FB,kto_kastory,kto_twt,kto_blog,organic
landing_cnt,8329.000000,5060.00000,1736.000000,1893.000000,164.00000,9.0,84.0,11.0,49.000000,1.865431e+06
contest_cnt,230.000000,215.00000,33.000000,13.000000,160.00000,9.0,84.0,11.0,46.000000,8.390000e+02
conversion,0.027614,0.04249,0.019009,0.006867,0.97561,1.0,1.0,1.0,0.938776,4.497620e-04


ver2


,FB_INSTA,KAKAO_moment,everytime,NAVER_GFA,organic,kto_sns
landing_cnt,8329.000000,5060.00000,1736.000000,1893.000000,1.865431e+06,317.000000
contest_cnt,230.000000,215.00000,33.000000,13.000000,8.390000e+02,310.000000
conversion,0.027614,0.04249,0.019009,0.006867,4.497620e-04,0.977918


### (4)Fraud
- 바운스율
- 정착율

In [92]:
#########################################
# 1. 바운스 유저 비율 & 정착 유저 비율
#########################################
dict_fraud_referrs = {}
dict_fraud_referrs2 = {}

def count_fraud(dict_input, dict_output, lst_fraud = lst_fraud_device):
    for _referrer, _segment in dict_input.items():
        _lst_device= list(set(_segment) & set(lst_fraud))       ## 수정된 부분
        
        total_cnt = len(_segment)
        fraud_cnt = len(_lst_device)
        settle_cnt = total_cnt - fraud_cnt
        fraud_rate = fraud_cnt / total_cnt
        settle_rate = 1 - fraud_rate                                

        dict_output[f'{_referrer}'] = [total_cnt, fraud_cnt, settle_cnt, fraud_rate, settle_rate]

    _df_output = pd.DataFrame.from_dict(
        dict_output, orient='index', columns=[
            'total_cnt', 'fraud_cnt', 'settle_cnt', 'fraud_rate', 'settle_rate']).sort_values(by = 'fraud_rate', ascending = False)
    
    return _df_output, dict_output

df_fraud_referrs, dict_fraud_referrs = count_fraud(dict_lst_user_from_referrs_gguonggu, dict_fraud_referrs)
df_fraud_referrs2, dict_fraud_referrs2 = count_fraud(dict_lst_user_from_referrs_gguonggu2, dict_fraud_referrs2)

print('ver1')
df_fraud_referrs

print('ver2')
df_fraud_referrs2

ver1


,total_cnt,fraud_cnt,settle_cnt,fraud_rate,settle_rate
everytime,1736,1496,240,0.861751,0.138249
NAVER_GFA,1893,1551,342,0.819334,0.180666
organic,1865431,1426365,439066,0.764630,0.235370
KAKAO_moment,5060,3627,1433,0.716798,0.283202
FB_INSTA,8329,5454,2875,0.654821,0.345179
kto_kastory,84,51,33,0.607143,0.392857
kto_twt,11,5,6,0.454545,0.545455
kto_blog,49,15,34,0.306122,0.693878
kto_ins,164,49,115,0.298780,0.701220
kto_FB,9,2,7,0.222222,0.777778


ver2


,total_cnt,fraud_cnt,settle_cnt,fraud_rate,settle_rate
everytime,1736,1496,240,0.861751,0.138249
NAVER_GFA,1893,1551,342,0.819334,0.180666
organic,1865431,1426365,439066,0.764630,0.235370
KAKAO_moment,5060,3627,1433,0.716798,0.283202
FB_INSTA,8329,5454,2875,0.654821,0.345179
kto_sns,317,122,195,0.384858,0.615142


### (5)앱설치 유저

In [89]:
# #########################################
# 0. prep
# #########################################
EVENT_APP_INSTALL = '#appInstall'
app_key_android = 'qsJfZhE2Ft85SvZoXInoo3'
app_key_ios = 'qsJfZhE2Ft85SvZTOvxsUA'

#########################################
# 1. 앱설치 유저
#########################################
dict_install = {}
dict_install2 = {}

def count_app_install(dict_input, dict_output):
    _df_log_install = df_app_log[df_app_log['abs_events'].apply(lambda x : True if EVENT_APP_INSTALL in x else False)]

    for _referrer, _segment in dict_input.items():
        ## 1) 캠페인 유저의 사용자 ID 추출
        _lst_campaign_user_id = list(set(df_prop[df_prop[KEY_ID_DEVICE].isin(_segment)][KEY_ID_USER]))
        
        ## 2) 캠페인 유저의 사용자 ID중 android & ios 앱 사용자
        dict_output[f'{_referrer}'] = {}
        _lst_campaign_user_ids =\
            list( 
                df_app_log[
                (df_app_log['app_key'].isin([app_key_android, app_key_ios])) & 
                (df_app_log[KEY_ID_USER].isin(_lst_campaign_user_id))
                ][KEY_ID_USER].unique()
                )
            # _df_log_install[_df_log_install[KEY_ID_USER].isin(_lst_user_id)][KEY_ID_USER]

        dict_output[f'{_referrer}'] = len(_lst_campaign_user_ids)

    df_output = pd.DataFrame.from_dict(
                    dict_output, orient='index', columns= ['install_cnt']).sort_values(by = 'install_cnt', ascending=False)

    return df_output, dict_output

df_install_referrs, dict_install = count_app_install(dict_lst_user_from_referrs_gguonggu, dict_install)
df_install_referrs2, dict_install2 = count_app_install(dict_lst_user_from_referrs_gguonggu2, dict_install2)

print('ver1')
df_install_referrs

print('ver2')
df_install_referrs2

ver1


,install_cnt
organic,7707
FB_INSTA,8
kto_ins,8
KAKAO_moment,5
kto_FB,2
kto_kastory,2
kto_twt,2
everytime,1
NAVER_GFA,1
kto_blog,1


ver2


,install_cnt
organic,7707
kto_sns,11
FB_INSTA,8
KAKAO_moment,5
everytime,1
NAVER_GFA,1


### (6)활성
- 인당 체류 시간
- 인당 컨텐츠 조회수
- 검색 조회 빈도

In [12]:
#########################################
# 1. 인당 체류시간
#########################################
dict_duration = {}
dict_events_cnt_output = {}
dict_duration2 = {}
dict_events_cnt_output2 = {}

def activation(dict_input, dict_ouput1, dict_output2, KEY_ID = KEY_ID_DEVICE):
    lst_device_no_fraud = list(set(lst_total_device) - set(lst_fraud_device))
    _df_log_no_fraud = df_app_log[df_app_log[KEY_ID].isin(lst_device_no_fraud)]

    for _referrer, _segment in dict_input.items():
        ## 1) 평균 체류 시간
        _df = _df_log_no_fraud[_df_log_no_fraud[KEY_ID_DEVICE].isin(_segment)]
        dict_ouput1[f'{_referrer}']= _df.groupby(KEY_ID)['duration'].sum().mean()

        ## 2) 평균 컨텐츠 조회수
        _df_app_log_by_referr = df_app_log[
                                    df_app_log[KEY_ID_DEVICE].isin(_segment)]
        _df_app_log_by_referr['events_cnt'] = _df_app_log_by_referr['abs_events'].apply(lambda x : len(x))

        dict_output2[f'{_referrer}'] = _df_app_log_by_referr.groupby(KEY_ID_DEVICE)['events_cnt'].sum().mean()

    ### 1-2) 평균 체류 시간 df화
    _df_output1 = pd.DataFrame.from_dict(dict_ouput1, orient='index', columns=['average_duration'])
    _df_output1 = _df_output1.sort_values(by = 'average_duration', ascending=False)

    ### 2-2) 평균 컨텐츠 조회수 df화
    _df_output2 = pd.DataFrame.from_dict(dict_output2, orient='index',  columns=['average_event_cnt']).sort_values(by = 'average_event_cnt', ascending=False)

    return _df_output1, _df_output2, lst_device_no_fraud

df_duration, df_event_cnt, lst_device_no_fraud = activation(dict_lst_user_from_referrs_gguonggu, dict_duration, dict_events_cnt_output)
df_duration2, df_event_cnt2, lst_device_no_fraud2 = activation(dict_lst_user_from_referrs_gguonggu2, dict_duration2, dict_events_cnt_output2)

print('ver1')
df_duration
df_event_cnt

print('ver2')
df_duration2
df_event_cnt2

ver1


,average_duration
kto_FB,2191.686143
kto_ins,1840.806435
kto_twt,932.074000
kto_blog,392.950353
organic,341.463692
everytime,284.049325
kto_kastory,230.866212
KAKAO_moment,187.402788
NAVER_GFA,160.201541
FB_INSTA,113.963729


,average_event_cnt
kto_FB,31.888889
kto_ins,16.176829
kto_twt,6.000000
kto_blog,3.775510
kto_kastory,2.380952
organic,2.037439
KAKAO_moment,1.648024
FB_INSTA,1.573538
NAVER_GFA,1.287903
everytime,1.254032


ver2


,average_duration
kto_sns,1300.542995
organic,341.463692
everytime,284.049325
KAKAO_moment,187.402788
NAVER_GFA,160.201541
FB_INSTA,113.963729


,average_event_cnt
kto_sns,10.697161
organic,2.037439
KAKAO_moment,1.648024
FB_INSTA,1.573538
NAVER_GFA,1.287903
everytime,1.254032
